In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# download sktime package 
!pip install sktime

     |████████████████████████████████| 5.9 MB 7.7 MB/s 
     |████████████████████████████████| 9.8 MB 43.0 MB/s 
     |████████████████████████████████| 23.2 MB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 26.3 MB 69 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
# 필요한 패키지 import
import os
import sys
import warnings
import plotly
import numpy as np
import pandas as pd
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler, StandardScaler
import datetime
import tensorflow as tf
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from scipy.stats import reciprocal 

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
%cd /content/gdrive/Shareddrives/cj공모전/traindata/시계열_item_cd/

/content/gdrive/Shareddrives/cj공모전/traindata/시계열_item_cd


In [ ]:
%ls

item_cd_138632647.pkl   item_cd_4802214590.pkl  item_cd_5326550997.pkl
item_cd_2055189125.pkl  item_cd_4824061918.pkl  item_cd_5404706313.pkl
item_cd_2108751155.pkl  item_cd_4904383364.pkl  item_cd_5480996007.pkl
item_cd_232155349.pkl   item_cd_4929712168.pkl  item_cd_5497354651.pkl
item_cd_3681859516.pkl  item_cd_4930635842.pkl  item_cd_5519470982.pkl
item_cd_414687646.pkl   item_cd_5002701828.pkl  item_cd_5521930034.pkl
item_cd_4207106945.pkl  item_cd_5002901812.pkl  item_cd_5534486731.pkl
item_cd_4288442391.pkl  item_cd_5102599276.pkl  item_cd_5569305322.pkl
item_cd_438888608.pkl   item_cd_5109058837.pkl  item_cd_5582257070.pkl
item_cd_447405954.pkl   item_cd_5121269742.pkl  item_cd_5636360821.pkl
item_cd_4609081354.pkl  item_cd_5157953843.pkl  item_cd_5638800672.pkl
item_cd_4610261043.pkl  item_cd_5187227890.pkl  item_cd_655045121.pkl
item_cd_4624200932.pkl  item_cd_5240328576.pkl  item_cd_657383604.pkl
item_cd_4658408144.pkl  item_cd_5240330199.pkl  item_cd_8809528221117.pkl
item_

In [ ]:
l0 = [ 5187227890, 655045121, 4624200932, 4610261043, 5326550997]

l1 = [ 5299553826, 8809641652409, 5519470982, 8809528221117, 8809528221414]

l2 = [ 4904383364, 4930635842, 5582257070, 5636360821, 5534486731]

l3 =[ 3681859516, 4929712168, 2055189125, 4207106945, 5157953843]

l4 =[ 2108751155, 5480996007, 5240328576, 5002901812, 5240330199]

l5 =[ 232155349, 657383604, 5404706313, 414687646, 438888608]

l6 = [ 5569305322, 138632647, 5102599276, 5121269742, 5521930034]

l7 =[ 5638800672, 5002701828, 5497354651, 4730793965, 447405954]

l8 =[ 4802177320, 4824061918, 4802214590, 4658408144, 4733964360]

l9 =[ 4609081354, 5109058837, 4288442391, 5279261934, 5279251727]

clusters = [l0, l1, l2, l3, l4, l5, l6, l7, l8, l9]

In [ ]:
# 학습에 필요한 train 및 test dataset 만드는 과정
def get_train_test_set(cd):
  df = pd.read_pickle("item_cd_"+cd+".pkl")
  train = df[df['BKG_DATE'] <= '2021-06-20']
  test = df[df['BKG_DATE'] > '2021-06-20']
  # 나머지 Scaling
  scaling_features = ['요일', '휴일여부', 'DAY_1', 'DAY_2', 'DAY_3', 'week', 'WEEK_AMT', 'MEAN_PRICE',
       '강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천',
       '전남', '전북', '제주', '충남', '충북']
  scaler = StandardScaler()
  train.loc[:, scaling_features] = scaler.fit_transform(train[scaling_features])
  test.loc[:, scaling_features] = scaler.transform(test[scaling_features])
  train_x = train.drop(['ITEM_QTY','BKG_DATE'], axis=1)
  train_y = train['ITEM_QTY']

  test_x = test.drop(['ITEM_QTY','BKG_DATE'], axis=1)
  test_y = test['ITEM_QTY']
  return train_x, train_y, test_x, test_y

In [ ]:
def get_stacking_base_datasets(model, x_train_n, y_train_n, x_test_n, n_splits=5):
  # 지정된 n_folds 값으로 KFold 생성
  tss = TimeSeriesSplit(n_splits)

  # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
  train_fold_pred=np.zeros((x_train_n.shape[0], 1))
  test_pred=np.zeros((x_test_n.shape[0], n_splits))
  #print(model.__class__.__name__, ' model 시작')

  for folder_counter, (train_index, valid_index) in enumerate(tss.split(x_train_n)):
    # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
    #print('\t 폴드 세트: ', folder_counter, ' 시작')
    x_tr=x_train_n[train_index]
    y_tr=y_train_n[train_index]
    x_te=x_train_n[valid_index]

    # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
    model.fit(x_tr, y_tr)
    # 폴드 세트 내부에서 다시 만들어지 검증 데이터로 기반 모델 예측 후 데이터 저장
    train_fold_pred[valid_index, :]=model.predict(x_te).reshape(-1, 1)
    # 입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
    test_pred[:, folder_counter]=model.predict(x_test_n)

  # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
  test_pred_mean=np.mean(test_pred, axis=1).reshape(-1, 1)

  # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
  return train_fold_pred, test_pred_mean

In [ ]:
# 모델별 학습 파라미터 정의
XGBRegressor_param = {'n_estimators' : [100], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1), 'max_depth' : np.arange(3,9,1) , 'colsample_bytree' :np.arange(0.8, 1.0, 0.1), 'subsample' :np.arange(0.8, 1.0, 0.1)}
LGBMRegressor_param = {'max_depth' : range(3,15,3), 'min_child_weight': range(1,6,2), 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100], 'learning_rate':[0.1, 0.01], 'max_depth' : [6,8,10]}
SVR_param = {'kernel':['linear'], 'C':[1.0], 'epsilon':[0.1]}
GradientBoostingRegressor_param = {'n_estimators':[100], 'max_depth':np.arange(3,20,3)}
AdaBoostRegressor_param = {'n_estimators' : np.arange(25, 100, 25), 'loss': ['linear', 'square', 'exponential'], 'learning_rate': np.arange(0.1, 1)} 

In [ ]:
# 모델별 정의
XGBRegressor_model = XGBRegressor(n_estimators = 100, objective = 'reg:squarederror')
LGBMRegressor_model = LGBMRegressor(n_estimators = 80)
SVR_model = SVR(kernel='linear', C=1.0, epsilon=0.1)
GradientBoostingRegressor_model = GradientBoostingRegressor(n_estimators=100, max_depth=3)
AdaBoostRegressor_model = AdaBoostRegressor(base_estimator=None)

In [ ]:
# 단일 모델별 최적 파라미터로 모델링
def print_best_params(model, params, x_train, x_test, y_train, y_test, log=False):
  tss = TimeSeriesSplit(n_splits=5)
  grid_model=GridSearchCV(model, cv = tss, param_grid=params, scoring='neg_mean_absolute_error')
  grid_model.fit(x_train, y_train)
  mae = -1 * grid_model.best_score_

  best_model=grid_model.best_estimator_
  pred=best_model.predict(x_test)

  if log:
    y_test=np.expm1(y_test)
    pred=np.expm1(pred)
  
  min_list = np.round(mean_absolute_error(y_test, pred), 4)

  return best_model, min_list, pred


하나의 샘플이긴 하더라도<br>

test MAE로 보았을 때<br>
XGBRegressor < GradientBoostingRegressor < SVR < AdaBoostRegressor < LGBMRegressor

In [ ]:
import sys

mod = sys.modules[__name__]

In [ ]:
# 최적의 파라미터로 각각의 모델 학습
def get_optimal_model(cd):
  x_train, y_train, x_test, y_test = get_train_test_set(cd)
  x_train_n=x_train.values
  x_test_n=x_test.values
  y_train_n=y_train.values
  
  # 단일 모델별 성능 리스트
  single_list = dict()

  # 단일 모델 학습
  globals()["XGBRegressor_model_tuned_{}".format(cd)], single_list["XGBRegressor"], globals()["single_pred_XGBRegressor_{}".format(cd)] = print_best_params(XGBRegressor_model, XGBRegressor_param, x_train, x_test, y_train, y_test)
  globals()["LGBMRegressor_model_tuned_{}".format(cd)], single_list["LGBMRegressor"], globals()["single_pred_LGBMRegressor_{}".format(cd)] = print_best_params(LGBMRegressor_model, LGBMRegressor_param, x_train, x_test, y_train, y_test)
  globals()["SVR_model_tuned_{}".format(cd)], single_list["SVR"], globals()["single_pred_SVR_{}".format(cd)] = print_best_params(SVR_model, SVR_param, x_train, x_test, y_train, y_test)
  globals()["GradientBoostingRegressor_model_tuned_{}".format(cd)], single_list["GradientBoostingRegressor"], globals()["single_pred_GradientBoostingRegressor_{}".format(cd)] = print_best_params(GradientBoostingRegressor_model, GradientBoostingRegressor_param, x_train, x_test, y_train, y_test)
  globals()["AdaBoostRegressor_model_tuned_{}".format(cd)], single_list["AdaBoostRegressor"], globals()["single_pred_AdaBoostRegressor_{}".format(cd)] = print_best_params(AdaBoostRegressor_model, AdaBoostRegressor_param, x_train, x_test, y_train, y_test)

  single_model_mae = sorted(single_list.items(), key = lambda item: item[1])

  # Stacking 모델별 성능 리스트  
  stacking_list = dict()

  # stacking model dataset 생성
  globals()["{}_train_{}".format(single_model_mae[0][0], cd)], globals()["{}_test_{}".format(single_model_mae[0][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cd)), x_train_n, y_train_n, x_test_n, 5)                                                                                        
  globals()["{}_train_{}".format(single_model_mae[1][0], cd)], globals()["{}_test_{}".format(single_model_mae[1][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cd)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            
  # 첫번째 경우
  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[0][0], cd)), getattr(mod, "{}_train_{}".format(single_model_mae[1][0], cd))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[0][0], cd)), getattr(mod, "{}_test_{}".format(single_model_mae[1][0], cd))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[2][0], cd)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cd))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[2][0], cd)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[2][0], cd)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[2][0], cd)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[2][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[2][0], cd)))

  # 두번째 경우
  globals()["{}_train_{}".format(single_model_mae[0][0], cd)], globals()["{}_test_{}".format(single_model_mae[0][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cd)), x_train_n, y_train_n, x_test_n, 5)                                                                                               
  globals()["{}_train_{}".format(single_model_mae[2][0], cd)], globals()["{}_test_{}".format(single_model_mae[2][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cd)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            

  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[0][0], cd)), getattr(mod, "{}_train_{}".format(single_model_mae[2][0], cd))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[0][0], cd)), getattr(mod, "{}_test_{}".format(single_model_mae[2][0], cd))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[1][0], cd)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cd))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[1][0], cd)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[1][0], cd)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[1][0], cd)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[1][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[1][0], cd)))


  # 세번째 경우
  globals()["{}_train_{}".format(single_model_mae[1][0], cd)], globals()["{}_test_{}".format(single_model_mae[1][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cd)), x_train_n, y_train_n, x_test_n, 5)
                                                                                                            
  globals()["{}_train_{}".format(single_model_mae[2][0], cd)], globals()["{}_test_{}".format(single_model_mae[2][0], cd)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cd)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            

  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[1][0], cd)), getattr(mod, "{}_train_{}".format(single_model_mae[2][0], cd))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[1][0], cd)), getattr(mod, "{}_test_{}".format(single_model_mae[2][0], cd))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[0][0], cd)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cd))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[0][0], cd)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[0][0], cd)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[0][0], cd)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[0][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[0][0], cd)))

  stacking_model_mae = sorted(stacking_list.items(), key = lambda item: item[1])

  # 단일 모델과 stacking 모델의 MAE 값을 비교하여 작은 값으로 모델링 결과 반환
  a = single_model_mae[0][1]
  b = stacking_model_mae[0][1]

  if a < b:
    print("Item 코드 {}의 최적 모델은 단일 모델 {} : (test MAE값) {}".format(cd, single_model_mae[0][0], single_model_mae[0][1]))
    globals()["best_model_{}".format(cd)]  = single_model_mae[0][0]
    globals()["test_mae_{}".format(cd)] = single_model_mae[0][1]
    globals()["best_pred_{}".format(cd)] = getattr(mod, "single_pred_{}_{}".format(single_model_mae[0][0], cd))
  else:
    print("Item 코드 {}의 최적 모델은 stacking meta 모델 {} : (test MAE값) {}".format(cd, stacking_model_mae[0][0], stacking_model_mae[0][1]))
    globals()["best_model_{}".format(cd)] = stacking_model_mae[0][0]
    globals()["test_mae_{}".format(cd)] = stacking_model_mae[0][1]
    globals()["best_pred_{}".format(cd)] = getattr(mod, "stack_pred_{}_{}".format(stacking_model_mae[0][0], cd))

In [ ]:
# 단일 모델과 stacking 모델의 MAE 값을 비교하여 작은 값으로 모델링 결과 반환
for i in range(len(clusters)):
  print("Cluster " + str(i))
  print("")
  for cd in clusters[i]:
    get_optimal_model(str(cd))
  print("__________________________________________________________________________")

Cluster 0

Item 코드 5187227890의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0393
Item 코드 655045121의 최적 모델은 단일 모델 GradientBoostingRegressor : (test MAE값) 2.5966
Item 코드 4624200932의 최적 모델은 stacking meta 모델 GradientBoostingRegressor : (test MAE값) 0.0009730483896049528
Item 코드 4610261043의 최적 모델은 단일 모델 SVR : (test MAE값) 0.1045
Item 코드 5326550997의 최적 모델은 단일 모델 SVR : (test MAE값) 0.3598
__________________________________________________________________________
Cluster 1

Item 코드 5299553826의 최적 모델은 단일 모델 SVR : (test MAE값) 12.1343
Item 코드 8809641652409의 최적 모델은 단일 모델 SVR : (test MAE값) 10.7657
Item 코드 5519470982의 최적 모델은 단일 모델 GradientBoostingRegressor : (test MAE값) 41.4856
Item 코드 8809528221117의 최적 모델은 단일 모델 SVR : (test MAE값) 6.7478
Item 코드 8809528221414의 최적 모델은 단일 모델 SVR : (test MAE값) 8.0534
__________________________________________________________________________
Cluster 2

Item 코드 4904383364의 최적 모델은 단일 모델 SVR : (test MAE값) 21.5987
Item 코드 4930635842의 최적 모델은 단일 모델 SVR : (test MAE값) 12.9045
Item 코드 55822570

In [ ]:
# test dataset에 대한 예측값 
for i in range(len(clusters)):
  globals()['item_pred_df_{}'.format(str(i))] = pd.DataFrame()
  for cd in clusters[i]:
    getattr(mod, 'item_pred_df_{}'.format(str(i)))[str(cd)] = getattr(mod, "best_pred_{}".format(cd))
  globals()['item_pred_df_{}'.format(str(i))] = getattr(mod, 'item_pred_df_{}'.format(str(i))).transpose()
  getattr(mod, 'item_pred_df_{}'.format(str(i)))['Cluster'] = i

In [ ]:
item_pred_df = pd.concat([item_pred_df_0, item_pred_df_1, item_pred_df_2, item_pred_df_3, item_pred_df_4, item_pred_df_5, item_pred_df_6, item_pred_df_7, item_pred_df_8, item_pred_df_9])

In [ ]:
item_pred_df.to_csv('/content/gdrive/Shareddrives/cj공모전/모델링/item_예측값.csv', encoding = 'utf-8')